In [1]:
from datasets import load_dataset

dataset = load_dataset("glue", "sst2")

text_column = "input_text"
label_column = "text_label"
max_length = 256

label_mapping = {0: "negative", 1: "positive"}

classes = dataset["train"].features["label"].names

dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1
)

## Add a prompt to each sentence
# init_prompt = """Classify the sentiment of the following sentence as positive or negative.
# Sentence: """

# dataset = dataset.map(
#     lambda x: {"input_text": [(init_prompt + text).strip() for text in x["sentence"]]},
#     batched=True,
#     num_proc=1
# )

dataset["train"][0]
# print(dataset["train"].features["label"].names)

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0,
 'text_label': 'negative'}

In [33]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/mnt/export/arun/llmenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
imdb_dataset = load_dataset("imdb")
label_mapping = {0: "negative", 1: "positive"}


In [42]:
financial_phrasebank = load_dataset("financial_phrasebank", "sentences_allagree")

/mnt/export/arun/llmenv/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [44]:
from tqdm import tqdm
prompt_template = """Classify the sentiment of the following sentence as "positive" or "negative".
Sentence: {text}
Answer: """

pred = []
true = []

for idx, item in tqdm(enumerate(financial_phrasebank["train"])):
    if item["label"] != 2:
        prompt = prompt_template.format(text=item["sentence"])
        inputs = tokenizer(prompt, return_tensors="pt", max_length=max_length, truncation=True)
        outputs = model.generate(**inputs)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        pred.append(prediction)
        true.append(label_mapping[item["label"]])



0it [00:00, ?it/s]/mnt/export/arun/llmenv/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2264it [03:33, 10.62it/s]


In [40]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true, pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9240759240759241


In [45]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true, pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8919716646989374


In [14]:
prompt = """Classify the sentiment of the following sentence as "positive" or "negative".
Sentence: I love this movie!"""

inputs = tokenizer(prompt, return_tensors="pt", max_length=max_length, truncation=True)
outputs = model.generate(**inputs)
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
prediction

'! Sentence: I love this movie! Sentence: I love this movie'

In [24]:
set(pred)

{'negative', 'positive'}

In [25]:
label_mapping = {0: "negative", 1: "positive"}
true = [label_mapping[label] for label in true]
set(true)

{'negative', 'positive'}

In [27]:
## Accuracy -- Accuracy on the Validation Set
from sklearn.metrics import accuracy_score

# label_mapping = {"fal": "negative", "true": "positive"}
# pred = [label_mapping[p.lower()] for p in pred]
accuracy_score(true, pred)

0.9506880733944955

In [32]:
!pip freeze | grep transformers

transformers==4.40.0


In [ ]:
import torch
from transformers import pipeline
hf_token = "HF_TOKEN_ID"
pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    device="cuda",
    token=hf_token 
)


In [ ]:
from tqdm import tqdm

prompt_template = """Classify the sentiment of the given sentence as positive or negative 
sentence: {sentence} 
Answer: """

for item in tqdm(dataset["validation"]):
    text = item["sentence"]
    prompt = prompt_template.format(sentence=text)
    output = pipe(prompt, max_length=4, num_return_sequences=1)
    print(output)
    break